# Graph Neural Network

## Importing

In [1]:
from selfdist_toolkit.pyg_tools import gnn_load, GIN_nn, execution, sd_utils, accuracy
import pandas as pd
import torch
import torch_geometric
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
from tqdm import tqdm

## Loading aid_list

In [2]:
aid_list = pd.read_csv("results/random_forest/experiments_check/chem-desc_good-aid_1.csv").aid.to_numpy().astype(int)

In [3]:
aid_list

array([    884,     891,     899,     914,    1418,    1431,    1770,
          1771,    1795,  493073,  493102,  493177,  493191,  493240,
        588834,  651741,  651812,  651814,  686978,  687022,  720691,
        743036,  743040,  743065, 1053173, 1259381, 1346982])

In [4]:
# for one aid now
aid = aid_list[0]
random_state = 131313

## Select mode: smooth for 2 dim label and hard for 1-dim label

In [5]:
# mode = "smooth"
mode = "hard"

## Load pytorch data

In [6]:
whole_data = gnn_load.load_pyg_data_aid(aid=aid, label_type=mode, do_in_parallel=True)

In [7]:
whole_data[:4]

[Data(x=[19, 9], edge_index=[2, 42], edge_attr=[42, 3], smiles='CC1(C=CC2=C(O1)C3=C(C=CC(=C3)OC)NC2=O)C', y=[1]),
 Data(x=[10, 9], edge_index=[2, 20], edge_attr=[20, 3], smiles='C1CN=C(N1)SCC(=O)O', y=[1]),
 Data(x=[29, 9], edge_index=[2, 64], edge_attr=[64, 3], smiles='CC(=O)OCC(=O)[C@]1(CC[C@@H]2[C@@]1(CC(=O)[C@H]3[C@H]2CCC4=CC(=O)CC[C@]34C)C)O', y=[1]),
 Data(x=[25, 9], edge_index=[2, 56], edge_attr=[56, 3], smiles='C[C@]12CCC(=O)C=C1CC[C@@H]3[C@@H]2CC[C@]4([C@H]3CC[C@@]4(C(=O)CO)O)C', y=[1])]

In [8]:
# build labels for split
labels_hard = np.array([
    data.y.detach().cpu().numpy()
    for data in whole_data
]).flatten().astype(int)

In [9]:
labels_hard.shape

(9593,)

In [10]:
labels_hard.sum()

3274

In [11]:
# data splitting
for train_idx, test_idx in StratifiedShuffleSplit(n_splits=1, random_state=random_state, test_size=0.2).split(whole_data, labels_hard):
    break

In [12]:
labels_hard[train_idx].sum()/len(train_idx)

0.34128225175918686

In [13]:
labels_hard[test_idx].sum()/len(test_idx)

0.341323606044815

In [14]:
# create the data loader
dl_train = torch_geometric.loader.DataLoader([whole_data[idx] for idx in train_idx], batch_size=100)
dl_test = torch_geometric.loader.DataLoader([whole_data[idx] for idx in test_idx], batch_size=100)

## Instantiate model

In [40]:
# GNN model
model = GIN_nn.GIN_basic(1)

In [41]:
# loss
loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([len(train_idx)/labels_hard[train_idx].sum()]))

In [42]:
# device
device = torch.device('cuda')

In [43]:
device

device(type='cuda')

In [44]:
model = model.to(device)
loss = loss.to(device)

In [45]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Train model

In [21]:
# training:
execution.training(model, dl_train, device, optimizer, loss)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 77/77 [00:04<00:00, 18.68it/s]


0.8904434

## Testing model

In [22]:
# testing:
pred_hard = execution.predict(model, dl_test, device, reduce_to_hard_label=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 74.32it/s]


In [23]:
pred_hard.sum()/pred_hard.shape[0]

0.5158936946326211

In [24]:
for batch in dl_train:
    break

In [25]:
batch

DataBatch(x=[2366, 9], edge_index=[2, 5100], edge_attr=[5100, 3], smiles=[100], y=[100], batch=[2366], ptr=[101])

In [26]:
batch = batch.to(device)

In [27]:
pred = model(batch).flatten()

In [28]:
loss.to(device)

BCEWithLogitsLoss()

In [29]:
loss(pred, batch.y)

tensor(0.8927, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [30]:
pred_hard[:5]

array([1, 1, 1, 1, 1])

In [31]:
# true labels
pred_hard_true = np.array([data.y.numpy().astype(int) for data in [whole_data[idx] for idx in test_idx]]).flatten()

In [32]:
pred_hard_true[:5]

array([0, 1, 0, 0, 1])

In [33]:
# get accuracy scores
acc = accuracy.calculate_accuracies_1d(y_true=pred_hard_true, y_pred=pred_hard)

In [34]:
acc.to_dict()

{'accuracy_score': 0.7222511724856696,
 'balanced_accuracy': 0.7527502657261571,
 'roc_score': 0.7527502657261571,
 'precision': 0.7801766069757506,
 'recall': 0.7222511724856696}

## Self distillation elements

In [35]:
# define number_to_pick
number_to_pick = 100

In [36]:
# load data for self distillation
selfdistillation_data = sd_utils.generate_gnn_sd_data(aid=aid)

In [78]:
# get good elements to insert into training pool for student model
selected, remaining = sd_utils.self_distillation_procedure_1dim(
    model=model, 
    self_distillation_data=selfdistillation_data, 
    number_to_pick=number_to_pick, 
    device=device, 
    correct_label=False, 
    batch_size=100
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4455/4455 [01:02<00:00, 71.14it/s]


In [79]:
selected[:5]

[Data(x=[5, 9], edge_index=[2, 8], edge_attr=[8, 3], smiles='OS(=O)(=O)O', y=[1]),
 Data(x=[11, 9], edge_index=[2, 20], edge_attr=[20, 3], smiles='C(P(=O)(O)O)(P(=O)(O)O)(Cl)Cl', y=[1]),
 Data(x=[4, 9], edge_index=[2, 6], edge_attr=[6, 3], smiles='Cl[Sb](Cl)Cl', y=[1]),
 Data(x=[9, 9], edge_index=[2, 18], edge_attr=[18, 3], smiles='C1(=O)NC(=O)NC(=O)N1', y=[1]),
 Data(x=[5, 9], edge_index=[2, 8], edge_attr=[8, 3], smiles='CS(=O)(=O)O', y=[1])]

In [80]:
label_analysis = np.array([data.y.numpy() for data in selected])

In [81]:
label_analysis.sum()/label_analysis.shape[0]

1.4962436580390205e-07

In [82]:
selected2, remaining2 = sd_utils.self_distillation_procedure_1dim_advanced(
    model=model, 
    self_distillation_data=selfdistillation_data, 
    number_to_pick=number_to_pick, 
    pos_elem_to_pick_ratio=0.3,
    device=device, 
    correct_label=False, 
    batch_size=100
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4455/4455 [01:00<00:00, 73.64it/s]


In [83]:
label_analysis2 = np.array([data.y.numpy() for data in selected2])

In [84]:
label_analysis2.sum()/label_analysis2.shape[0]

0.29977272033691404

In [ ]:
# todo: ausbauen auf viele epochen und student - teacher übergang. außerdem begutachten ob positive oder negative label bevorzugt werden sollten

## Whole iteration for a certain number of epochs

In [53]:
# GNN model
model = GIN_nn.GIN_basic(1)
# loss
loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([len(train_idx)/labels_hard[train_idx].sum()]))
# device
device = torch.device('cuda')
model = model.to(device)
loss = loss.to(device)
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [54]:
# define number of epochs
epochs = 100

In [55]:
# define loss saving space
loss_storage = []

In [56]:
# define accuracy storage
accuracy_storage = []

In [57]:
# iterate over epochs
for epoch in tqdm(range(epochs)):
    
    # do training step
    loss_value = execution.training(model, dl_train, device, optimizer, loss, verbose=False)
    
    # put loss into storage
    loss_storage.append(loss_value)
    
    # do testing step
    y_pred_hard = execution.predict(model, dl_test, device, reduce_to_hard_label=True, verbose=False)
    
    # calculate accuracy DataFrame
    accuracy_storage.append(
        # get accuracy class and from it the DataFrame
        accuracy.calculate_accuracies_1d(
            y_true=y_pred_hard, 
            y_pred=accuracy.helper_pyg_to_numpy_label(
                data_loader=dl_test
            )
        ).to_df(index=epoch)
    )

# transform accuracy storage to pandas df
accuracy_storage = pd.concat(accuracy_storage)

# add new column for loss
accuracy_storage['loss'] = loss_storage

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:28<00:00,  3.28s/it]


In [58]:
accuracy_storage

,accuracy_score,balanced_accuracy,roc_score,precision,recall,loss
0,0.706097,0.701868,0.701868,0.722720,0.706097,0.893526
1,0.659719,0.729410,0.729410,0.800247,0.659719,0.795971
2,0.681084,0.739103,0.739103,0.802563,0.681084,0.757377
3,0.678999,0.741389,0.741389,0.807920,0.678999,0.742832
4,0.722772,0.748719,0.748719,0.792274,0.722772,0.716187
...,...,...,...,...,...,...
95,0.862949,0.844688,0.844688,0.862926,0.862949,0.305865
96,0.862428,0.844155,0.844155,0.862766,0.862428,0.308854
97,0.858260,0.839946,0.839946,0.859370,0.858260,0.298310
98,0.858781,0.840634,0.840634,0.860456,0.858781,0.296967
